In [9]:
import pandas

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

# Variable declarations
FILE_PATH = "Filtered_Data.csv"
SEPARATOR = ","
CLICK_THRESHOLD = 1000
MAX_ROWS = 500
MISCELLANEOUS = 'misc'
FEATURE_NAME = ['ad_advertiser_id', 'analyzer_name', 'click_browser', 'enriched_derived_device', 'enriched_derived_os', 'raw_sync_partner_id']

# imported the CSV data
dataframe = pandas.read_csv(FILE_PATH, sep = SEPARATOR)

# Drop the unused column
if 'Unnamed: 0' in dataframe.columns:
    dataframe.drop('Unnamed: 0', axis = 1, inplace = True)

# Remove the same prefix from all columns names
dataframe.columns  = [ column_name.split('.')[1] for column_name in dataframe.columns ]

print (dataframe.head())
print (dataframe.shape[0], dataframe.shape[1])
CLICK_THRESHOLD = max(1000, int(float(dataframe.shape[0]) * .01))
print (CLICK_THRESHOLD)

         ts impression_system_date  hour   ...     kmean_category_name  partner_id  version
0  20190106             2019-01-06     1   ...                 missing   8PRHGG6T9        1
1  20190106             2019-01-06     1   ...                 missing   8PRHGG6T9        1
2  20190106             2019-01-06     2   ...                 missing   8PRHGG6T9        1
3  20190106             2019-01-06     4   ...                 missing   8PRHGG6T9        1
4  20190106             2019-01-06     6   ...                 missing   8PRHGG6T9        1

[5 rows x 42 columns]
124335 42
1243


In [47]:
# removing all data having single values in entire columns
for column in dataframe.columns:
    if(dataframe[column].nunique() == 1):
        dataframe.drop(column, axis = 1, inplace = True)

In [48]:
# Remove the timestamp data and impression system date data - results not dependent on the time frame
dataframe.drop('ts', axis = 1, inplace = True)
dataframe.drop('impression_system_date', axis = 1, inplace = True)

# Server hour do not matters
dataframe.drop('hour', axis = 1, inplace = True)

# Data have only 3 values of *au* region, and conversion status 0 for all of them
dataframe.drop('enriched_country', axis = 1, inplace = True)

# Data is similar to cvr_base_data_table.click_city
dataframe.drop('impression_city', axis = 1, inplace = True)

# Ad-Bid is the cost which is irrespecti   ve of conversion from user side
dataframe.drop('ad_bid', axis = 1, inplace = True)

# Ad CPA goal do not affects the conversion status
dataframe.drop('ad_cpa_goal', axis = 1, inplace = True)

# Click Conf GMT Offset do not affects the conversion status
dataframe.drop('click_conf_gmt_offset', axis = 1, inplace = True)

# Date Time do not affects and is too granuler to looku[, thus do not affects the conversion status
dataframe.drop('click_user_datetime', axis = 1, inplace = True)

# Publisher Domain do not affects the conversion status
dataframe.drop('enriched_publisher_domain', axis = 1, inplace = True)

# Impression ISP Name do not affects the conversion status
dataframe.drop('impression_isp_name', axis = 1, inplace = True)

# below data is too granular to predict something
dataframe.drop('ad_keyword', axis = 1, inplace = True)
dataframe.drop('click_city', axis = 1, inplace = True)
dataframe.drop('raw_sub_publisher_id', axis = 1, inplace = True)
dataframe.drop('category_name', axis = 1, inplace = True)
dataframe.drop('click_id', axis = 1, inplace = True)
dataframe.drop('cc_id', axis = 1, inplace = True)
dataframe = dataframe.drop('raw_publisher_id', axis = 1, inplace = True)
dataframe = dataframe.drop('raw_sub_sub_publisher_id', axis = 1, inplace = True)

In [49]:
dataframe.head()

,conversion_status,ad_id,hour_of_day,enriched_derived_device,enriched_derived_os,ad_adgroup_id,ad_advertiser_id,ad_campaign_id,impression_state,impression_asn_code,impression_connection_type,analyzer_name,click_browser,raw_sync_partner_id,click_state,partner_id
0,1,138154,20,mobile,android,94663,1518,86975,ga,22394,mobile,keywordanalyserv2,"google,chrome",1,ga,8PRHGG6T9
1,1,138154,20,mobile,android,94663,1518,86975,ga,22394,mobile,keywordanalyserv2,"google,chrome",1,ga,8PRHGG6T9
2,0,138154,20,mobile,android,94663,1518,86975,al,21928,mobile,keywordanalyserv2,"google,chrome",1,al,8PRHGG6T9
3,0,138154,23,mobile,android,94663,1518,86975,in,7922,cable,keywordanalyserv2,"google,chrome",1,in,8PRHGG6T9
4,0,128699,22,mobile,android,81679,1458,73823,ca,15169,broadband,keywordanalyserv2,"google,chrome",1,ca,8PRHGG6T9


In [50]:
# Following column names have less entries thus can be relabelled
total = dataframe.shape[0]
pandas.set_option('display.max_rows', MAX_ROWS)

# Function for aggregation on basis of click, conversion and CVR(conversion ratio)
func = {'conversion_status' : {
    'conversion' : 'sum',
    'click' : 'count'
}}

# Iterate through above columns list and 'misc' the data which is too sparse to use 
# for classification or modelling
for col in FEATURE_NAME:
    dfg = dataframe.groupby(col).agg(func).reset_index()
    dfg.columns = dfg.columns.droplevel(0)
    dfg['cvr'] = dfg['conversion'] / dfg['click']
    dfg['click percentage of total data'] = dfg['click'] / total * 100
    declareMisc = set()
    for index, row in dfg.iterrows():
        if(row['click'] >= CLICK_THRESHOLD):
            continue
        declareMisc.add(row[''])
    print (declareMisc)
    for vals in declareMisc:
        dataframe.loc[dataframe[col] == vals, col] = MISCELLANEOUS
    
    print (dataframe[col].value_counts(), '\n\n')

{1633.0, 1351.0}
1387    30042
1458    21280
1637    13763
1608    10848
1619     7491
1640     5422
1581     5397
1461     4666
1375     4468
1518     3869
1454     3199
1352     3192
1492     2985
1521     2000
1517     1890
1641     1500
1630     1478
misc      845
Name: ad_advertiser_id, dtype: int64 


{'broadanalyserv2', 'categoryanalyser'}
keywordanalyserv2     104031
nofilteranalyser        8809
phraseanalyser          7537
forcedanalyser          1904
contextualanalyser      1469
misc                     585
Name: analyzer_name, dtype: int64 


{'blackberry,browser', 'nokia', 'ie,mobile', 'mozilla,like,unknown', 'opera,mini', 'opera', 'android,httpurlconnection', 'android,browser', 'unknown', 'chromium', 'ucbrowser', 'maxthon'}
google,chrome        61376
safari               25944
edge                 16964
internet,explorer    11315
mozilla,firefox       3600
applewebkit           3413
silk                  1356
misc                   367
Name: click_browser, dtype: int64 




In [51]:
dataframe.shape

(124335, 16)